In [1]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_weiwei.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [2]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,alpha,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            #print(rfe_score)
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            if len(candi_fea)==optim_featur_num:
                return candi_fea

In [3]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,0.5)

In [4]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8131765749778171, 0.8095999269272927, 0.8121496424656819, 0.8162338848582911, 0.8141917636619865, 0.81061120100214, 0.8070384675609376, 0.8101088261391514, 0.8131713554987213, 0.8131713554987213, 0.8111266245628685, 0.8101062163996033, 0.812147032726134, 0.810616420481236, 0.8106177253510098, 0.813681559580354, 0.8111344537815125, 0.8126676757659584, 0.8070449919098074, 0.8131739652382693, 0.8121509473354559, 0.812144422986586, 0.81214572785636, 0.8136776449710318, 0.8106112010021398, 0.8100996920507335, 0.8111214050837727, 0.812144422986586, 0.8126559319379926, 0.8126572368077666, 0.8121470327261339, 0.8116368286445013, 0.8147019677436192, 0.8136802547105798, 0.8157197661673365, 0.8136763401012578, 0.8152056474763819, 0.8126559319379926, 0.81061120100214, 0.812147032726134, 0.8101049115298293] [0.7016745757634548, 0.677793367

In [5]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8131765749778171, 0.8095999269272927, 0.8121496424656819, 0.8162338848582911, 0.8141917636619865, 0.81061120100214, 0.8070384675609376, 0.80754867164257, 0.8101023017902813, 0.8121483375959079, 0.8111227099535467, 0.8095973171877446, 0.8111305391721906, 0.8136828644501278, 0.8106203350905581, 0.8126624562868626, 0.8111344537815125, 0.8116433529933713, 0.8126611514170886, 0.8126637611566366, 0.8136880839292239, 0.8136867790594499, 0.8080627903335248, 0.8146993580040711, 0.8106112010021398, 0.8101023017902815, 0.8116316091654052, 0.8100970823111855, 0.8095881830993266, 0.8116316091654052, 0.8141865441828905, 0.8126559319379926, 0.8131674408893993, 0.8131674408893993, 0.8157197661673365, 0.8136763401012578, 0.8152056474763819, 0.8126559319379926, 0.81061120100214, 0.812147032726134, 0.8101049115298293] [0.7016745757634548, 0.6777

In [6]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8131765749778171, 0.8095999269272927, 0.8121496424656819, 0.8162338848582911, 0.8141917636619865, 0.8131700506289473, 0.8126546270682187, 0.8111227099535467, 0.8101023017902813, 0.8095920977086488, 0.8111240148233205, 0.8111240148233207, 0.8106177253510098, 0.813677644971032, 0.8147032726133931, 0.8136802547105798, 0.8126585416775406, 0.8116433529933713, 0.8126611514170886, 0.8126637611566366, 0.8136880839292239, 0.8136867790594499, 0.8080627903335248, 0.8146993580040711, 0.8106112010021398, 0.8101023017902815, 0.8116316091654052, 0.8100970823111855, 0.8095881830993266, 0.8116316091654052, 0.8121405083772639, 0.8126559319379926, 0.8131674408893993, 0.8131674408893993, 0.810613810741688, 0.8136763401012578, 0.8152056474763819, 0.8126559319379926, 0.81061120100214, 0.812147032726134, 0.8101049115298293] [0.7016745757634548, 0.67

In [7]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8131765749778171, 0.8095999269272927, 0.8121496424656819, 0.8162338848582911, 0.8141917636619865, 0.8131700506289473, 0.8131713554987213, 0.810612505871914, 0.810616420481236, 0.8095920977086488, 0.8111240148233205, 0.8111240148233207, 0.8106177253510098, 0.813677644971032, 0.8147032726133931, 0.8152160864345739, 0.8147045774831672, 0.8111279294326426, 0.8126611514170886, 0.8126637611566366, 0.8136880839292239, 0.8136867790594499, 0.8141956782713086, 0.8116355237747273, 0.810613810741688, 0.8070384675609373, 0.8116316091654052, 0.8116303042956314, 0.8095881830993266, 0.8116316091654052, 0.8121405083772639, 0.8116329140351792, 0.8080562659846547, 0.8095881830993266, 0.810613810741688, 0.8136763401012578, 0.8152056474763819, 0.8126559319379926, 0.81061120100214, 0.812147032726134, 0.8101049115298293] [0.7016745757634548, 0.67779

In [8]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8131765749778171, 0.8095999269272927, 0.8055169894044575, 0.8126598465473146, 0.8136815595803538, 0.8131700506289473, 0.8131713554987213, 0.810612505871914, 0.810616420481236, 0.8095920977086488, 0.8111240148233205, 0.8111240148233207, 0.8106177253510098, 0.8141917636619865, 0.8152147815647999, 0.8152160864345739, 0.8147045774831672, 0.8111279294326426, 0.8126611514170886, 0.8126637611566366, 0.8136880839292239, 0.8131752701080431, 0.8106203350905581, 0.8136854741896759, 0.8147084920924893, 0.8090831984967901, 0.8090818936270161, 0.810612505871914, 0.8065243488699828, 0.8116316091654052, 0.8121405083772639, 0.8116329140351792, 0.8080562659846547, 0.8095881830993266, 0.810613810741688, 0.8095920977086486, 0.8101023017902813, 0.8095920977086488, 0.81061120100214, 0.812147032726134, 0.8101049115298293] [0.7016745757634548, 0.6777

In [9]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8106203350905579, 0.8090884179758859, 0.8055169894044575, 0.8050054804530508, 0.8065347878281747, 0.8085769090244794, 0.8101127407484734, 0.8106216399603319, 0.8070423821702594, 0.8075538911216661, 0.8085756041547054, 0.8111240148233207, 0.8106177253510098, 0.8106151156114618, 0.8131687457591733, 0.8162299702489693, 0.8167480035492458, 0.812150947335456, 0.8070436870400334, 0.8126637611566366, 0.8136880839292239, 0.8131752701080431, 0.8106203350905581, 0.8136854741896759, 0.8147084920924893, 0.8065347878281747, 0.8090818936270161, 0.810612505871914, 0.8111214050837727, 0.8080549611148807, 0.8121405083772639, 0.8116329140351792, 0.8080562659846547, 0.8095881830993266, 0.810613810741688, 0.8095920977086486, 0.8101023017902813, 0.8095920977086488, 0.81061120100214, 0.812147032726134, 0.8101049115298293] [0.7016745757634548, 0.677

In [10]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8106203350905579, 0.8090884179758859, 0.8055169894044575, 0.8055169894044575, 0.8034722584686047, 0.8106229448301059, 0.8101127407484734, 0.8106216399603319, 0.8070423821702594, 0.8075538911216661, 0.8085756041547054, 0.8111240148233207, 0.8106177253510098, 0.8106151156114618, 0.810616420481236, 0.8147019677436191, 0.8167480035492458, 0.8157236807766586, 0.8157249856464326, 0.817767106842737, 0.8172582076308783, 0.8141930685317605, 0.8106203350905581, 0.8136854741896759, 0.8147084920924893, 0.8131726603684951, 0.8095920977086488, 0.810612505871914, 0.8090792838874681, 0.8085729944151574, 0.8080640952032987, 0.8101075212693771, 0.8080667049428467, 0.8111227099535467, 0.8090766741479201, 0.8095920977086486, 0.8101023017902813, 0.8095920977086488, 0.8101023017902815, 0.8080614854637508, 0.8101049115298293] [0.7016745757634548, 0.

In [11]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.785603371783496, 0.8050028707135027, 0.8070489065191294, 0.8080706195521687, 0.8096025366668407, 0.8106203350905579, 0.8106216399603319, 0.8106190302207841, 0.8055169894044575, 0.8070489065191294, 0.8070436870400334, 0.8080654000730727, 0.8070449919098074, 0.8070423821702594, 0.8075538911216661, 0.8060232788767682, 0.8075538911216661, 0.8111240148233207, 0.8111292343024166, 0.8106164204812358, 0.8116355237747273, 0.8111279294326426, 0.8080627903335248, 0.8065256537397568, 0.8070397724307113, 0.8080588757242027, 0.8070384675609376, 0.8116342189049532, 0.8116355237747273, 0.8090818936270161, 0.8080601805939768, 0.8111279294326426, 0.8101088261391514, 0.8095960123179706, 0.8034683438592829, 0.8080640952032987, 0.8101075212693771, 0.8080667049428467, 0.8085742992849314, 0.8090766741479201, 0.8111240148233205, 0.8101023017902813, 0.8095920977086488, 0.8101023017902815, 0.8080614854637508, 0.8101049115298293] [0.7016745757634548,

In [12]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.5405749256224228, 0.5594890129965029, 0.7876481027193487, 0.7794770081945821, 0.7815230440002087, 0.8014301372723003, 0.8070476016493554, 0.8101101310089254, 0.8101088261391514, 0.8106216399603319, 0.8075525862518921, 0.8085716895453834, 0.8070423821702594, 0.8085756041547054, 0.8106164204812358, 0.8116368286445013, 0.8111227099535465, 0.8055117699253614, 0.8065321780886269, 0.8095907928388748, 0.8101036066600553, 0.8095934025784228, 0.8070345529516153, 0.8080562659846547, 0.8065243488699828, 0.8080614854637508, 0.8085651651965134, 0.8085651651965133, 0.8080536562451067, 0.8095894879691006, 0.8070384675609376, 0.8080614854637508, 0.8065295683490789, 0.8075525862518921, 0.8080627903335248, 0.8111279294326426, 0.8101049115298293, 0.8075512813821181, 0.8070410773004854, 0.8101075212693771, 0.8080667049428467, 0.8085742992849314, 0.8090766741479201, 0.8055065504462654, 0.8090845033665639, 0.8095947074481966, 0.812147032726134, 0.8080614854637508, 0.8101049115298293] [0.7016745757634548,

In [26]:
np.max(aver_mean_lis1[:16]),np.max(aver_mean_lis2[:16]),np.max(aver_mean_lis3[:16]),np.max(aver_mean_lis4[:16]),np.max(aver_mean_lis5[:16])

(0.8162338848582911,
 0.8162338848582911,
 0.8162338848582911,
 0.8162338848582911,
 0.8136815595803538)

In [27]:
np.max(aver_mean_lis6[:16]),np.max(aver_mean_lis7[:16]),np.max(aver_mean_lis8[:16]),np.max(aver_mean_lis9[:16])

(0.8106216399603319,
 0.8106229448301059,
 0.8106216399603319,
 0.8116368286445013)

In [29]:
np.argmax(aver_mean_lis1[:16]),np.argmax(aver_mean_lis2[:16]),np.argmax(aver_mean_lis3[:16]),np.argmax(aver_mean_lis4[:16])

(11, 11, 11, 11)

In [105]:
aver_mean_lis1[:10]

[0.5405749256224228,
 0.5594890129965029,
 0.7876481027193487,
 0.785603371783496,
 0.8050028707135027,
 0.8070489065191294,
 0.8080706195521687,
 0.8096025366668407,
 0.8131765749778171,
 0.8095999269272927]

# SVM-RFE

In [13]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        kf=KFold(n_splits=5,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            if len(candi_fea)==featur_num:
                break
        return candi_fea

In [14]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.5)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [15]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.5405749256224228,
  0.5594890129965029,
  0.7876481027193487,
  0.785603371783496,
  0.8050028707135027,
  0.8070489065191294,
  0.8080706195521687,
  0.8096025366668407,
  0.8131765749778171,
  0.8095999269272927,
  0.8121496424656819,
  0.8162338848582911,
  0.8141917636619865,
  0.81061120100214,
  0.8070384675609376,
  0.80754867164257,
  0.8116368286445013,
  0.8136815595803538,
  0.8126585416775406,
  0.8157236807766586,
  0.8157236807766586,
  0.8157236807766586,
  0.8167466986794718,
  0.8182733963150479,
  0.8136802547105798,
  0.8157262905162064,
  0.8147045774831672,
  0.8131791847173652,
  0.8147071872227152,
  0.8152108669554778,
  0.8136802547105798,
  0.8152160864345739,
  0.8141930685317605,
  0.8167414792003758,
  0.8141865441828905,
  0.8111279294326426,
  0.8116368286445013,
  0.8131687457591733,
  0.812144422986586,
  0.8131674408893993,
  0.8131687457591733,
  0.812148337595908,
  0.8106151156114618,
  0.8116355237747273,
  0.81061120100214,
  0.808570384675609

# SVM-RELIEF

In [16]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(kernel='linear',C=self.C)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [17]:
relief_SVM_model=relief_SVM(x_train,y_train,0.5)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [18]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.7789602797640796,
  0.7901925987786418,
  0.7927449240565791,
  0.7912130069419072,
  0.7917245158933139,
  0.7917219061537659,
  0.7963254867164257,
  0.8095960123179706,
  0.8070462967795814,
  0.8050041755832767,
  0.8065373975677227,
  0.8070476016493554,
  0.7998982201576282,
  0.7983649981731823,
  0.7968265567096403,
  0.801937731614385,
  0.798862153557075,
  0.7947739965551439,
  0.7942624876037371,
  0.7942637924735112,
  0.7978417453938096,
  0.7937561981314264,
  0.801920768307323,
  0.800390156062425,
  0.800388851192651,
  0.7993723576387076,
  0.7963111331489117,
  0.7973315413121771,
  0.7988634584268489,
  0.7993749673782556,
  0.7993736625084816,
  0.7978456600031317,
  0.7978482697426796,
  0.7952894201158724,
  0.7968239469700924,
  0.7968226421003184,
  0.7993762722480297,
  0.7973302364424031,
  0.7947779111644657,
  0.7963085234093638,
  0.7983519494754423,
  0.798862153557075,
  0.7968213372305444,
  0.7978430502635836,
  0.7968187274909964,
  0.794776606294

In [19]:
weiwei_accuracy=[]
weiwei_accuracy.append(aver_mean_lis1)
weiwei_accuracy.append(aver_mean_lis2)
weiwei_accuracy.append(aver_mean_lis3)
weiwei_accuracy.append(aver_mean_lis4)
weiwei_accuracy.append(aver_mean_lis5)
weiwei_accuracy.append(aver_mean_lis6)
weiwei_accuracy.append(aver_mean_lis7)
weiwei_accuracy.append(aver_mean_lis8)
weiwei_accuracy.append(aver_mean_lis9)
weiwei_accuracy.append(rfe_SVM_aver_mean_lis)
weiwei_accuracy.append(relief_SVM_aver_mean_lis)

In [20]:
colu=[i for i in range(1,50)]
weiwei_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(weiwei_accuracy))
weiwei_acurracy_dataframe.to_csv("weiwei_accuracy.csv")

In [21]:
weiwei_F_measure=[]
weiwei_F_measure.append(F_measure_lis1)
weiwei_F_measure.append(F_measure_lis2)
weiwei_F_measure.append(F_measure_lis3)
weiwei_F_measure.append(F_measure_lis4)
weiwei_F_measure.append(F_measure_lis5)
weiwei_F_measure.append(F_measure_lis6)
weiwei_F_measure.append(F_measure_lis7)
weiwei_F_measure.append(F_measure_lis8)
weiwei_F_measure.append(F_measure_lis9)
weiwei_F_measure.append(rfe_SVM_F_measure_lis)
weiwei_F_measure.append(relief_SVM_F_measure_lis)

In [22]:
colu=[i for i in range(1,50)]
weiwei_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(weiwei_F_measure))
weiwei_F_measure_dataframe.to_csv("weiwei_F_measure.csv")